In [ ]:
import numpy

file_name_test='testTrack_hierarchy.txt'
file_name_train='trainIdx2_matrix.txt'
output_file='output.txt'

fTest= open(file_name_test, 'r')
fTrain=open(file_name_train, 'r')
Trainline= fTrain.readline()
fOut = open(output_file, 'w')

trackID_vec=[0]*6
albumID_vec=[0]*6
artistID_vec=[0]*6
lastUserID=-1

user_rating_inTrain=numpy.zeros(shape=(6,3))
for line in fTest:
	arr_test=line.strip().split('|')
	userID= arr_test[0]
	trackID= arr_test[1]
	albumID= arr_test[2]
	artistID=arr_test[3]

	if userID!= lastUserID:
		ii=0
		user_rating_inTrain=numpy.zeros(shape=(6,3))

	trackID_vec[ii]=trackID
	albumID_vec[ii]=albumID
	artistID_vec[ii]=artistID
	ii=ii+1
	lastUserID=userID

	if ii==6 : 
		while (Trainline):
			arr_train = Trainline.strip().split('|')
			trainUserID=arr_train[0]
			trainItemID=arr_train[1]
			trainRating=arr_train[2]
			Trainline=fTrain.readline()		

			if trainUserID< userID:
				continue
			if trainUserID== userID:				
				for nn in range(0, 6):
					if trainItemID==albumID_vec[nn]:
						user_rating_inTrain[nn, 0]=trainRating
					if trainItemID==artistID_vec[nn]:
						user_rating_inTrain[nn, 1]=trainRating
			if trainUserID> userID:
				for nn in range(0, 6):
					outStr=str(userID) + '|' + str(trackID_vec[nn])+ '|' + str(user_rating_inTrain[nn,0]) + '|' + str(user_rating_inTrain[nn, 1])
					fOut.write(outStr + '\n')
				break

fTest.close()
fTrain.close()

In [ ]:
import pandas as pd

output = pd.read_csv('output.txt',sep='|',header=None)

output.columns=['UserID','TrackID','Album Rating','Artist Rating']

In [ ]:
output['Mean']=output[['Album Rating','Artist Rating']].mean(axis=1)

train_df=pd.read_csv('trainIdx2_matrix.txt', sep='|',header=None)
train_df.columns=['trainUserID','trainItemID','trainRating']

prediction_df = output.copy()
prediction_df['Predictor'] = 0

#Sort by userID, then prediction
prediction_df = prediction_df.sort_values(by=['UserID','Mean'])

# first 3 prediction_df per user assign 0, and second 3 assign 1
count = 0
for ind in prediction_df.index:
  if(count < 3):
    prediction_df['Predictor'][ind] = 0
    count += 1
  elif (5 > count >= 3):
    prediction_df['Predictor'][ind] =  1
    count += 1
  elif (count == 5):
    prediction_df['Predictor'][ind] = 1
    count = 0

# combine user id and track id
prediction_df['TrackID'] = prediction_df['UserID'].astype(str) + '_' + prediction_df['TrackID'].astype(str)
prediction_df['TrackID'] = prediction_df['TrackID'].astype(str)
prediction_df = prediction_df.drop(['UserID', 'Album Rating', 'Artist Rating', 'Mean'], axis=1)

# save dataframe to a single csv file 
prediction_df.to_csv('prediction_mean.csv', index = False)
prediction_df.tail()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,TrackID,Predictor
119996,249010_86104,0
119998,249010_293818,0
119994,249010_110470,1
119997,249010_186634,1
119999,249010_262811,1
